In [1]:
import icechunk
import numpy as np
import zarr

In [2]:
repo = icechunk.Repository.create(icechunk.in_memory_storage())
session = repo.writable_session(branch="main")
root = zarr.group(session.store)
root.attrs["foo"] = "bar"
root.create_array("data", shape=(10, 10), chunks=(1, 1), dtype=np.int32)
session.commit(message="Add foo attribute and data array")

'98VJK6SE2G35EPVCRK3G'

In [3]:
session1 = repo.writable_session(branch="main")
session2 = repo.writable_session(branch="main")

root1 = zarr.group(session1.store)
root2 = zarr.group(session2.store)

root1.attrs["foo"] = "bar"
root2.attrs["foo"] = "baz"

print(session1.commit(message="Update foo attribute on root group"))
print(session2.commit(message="Update foo attribute on root group"))


7KH81ACN3YQ1SKYN3GE0


ConflictError: Failed to commit, expected parent: Some("98VJK6SE2G35EPVCRK3G"), actual parent: Some("7KH81ACN3YQ1SKYN3GE0")

In [4]:
session2.rebase(icechunk.ConflictDetector())

RebaseFailedError: Rebase failed on snapshot 7KH81ACN3YQ1SKYN3GE0: 1 conflicts found

In [5]:
try:
    session2.rebase(icechunk.ConflictDetector())
except icechunk.RebaseFailedError as e:
    print(e.conflicts)


[Conflict(UserAttributesDoubleUpdate, path=/)]


In [6]:
session2.rebase(icechunk.BasicConflictSolver(on_user_attributes_conflict=icechunk.VersionSelection.UseOurs))

In [7]:
session2.commit(message="Update foo attribute on root group")

'NXZY2WJNS5T15EXAPFBG'

In [8]:
session1 = repo.writable_session(branch="main")
session2 = repo.writable_session(branch="main")

root1 = zarr.group(session1.store)
root2 = zarr.group(session2.store)

root1["data"][0,0] = 1
root2["data"][0,:] = 2

In [9]:
print(session2.commit(message="Update first row of data array"))
print(session1.commit(message="Update first element of data array"))


AHSAVG18T7JMDE433Z0G


ConflictError: Failed to commit, expected parent: Some("NXZY2WJNS5T15EXAPFBG"), actual parent: Some("AHSAVG18T7JMDE433Z0G")

In [10]:
try:
    session1.rebase(icechunk.ConflictDetector())
except icechunk.RebaseFailedError as e:
    for conflict in e.conflicts:
        print(f"Conflict at {conflict.path}: {conflict.conflicted_chunks}")

Conflict at /data: [[0, 0]]


In [11]:
session1.rebase(icechunk.BasicConflictSolver(on_chunk_conflict=icechunk.VersionSelection.UseOurs))
session1.commit(message="Update first element of data array")

'7JG0TDW9SM3GEA9E9WY0'

In [12]:
session = repo.readonly_session(branch="main")
root = zarr.open_group(session.store, mode="r")
root["data"][0,:]

array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [13]:
session1 = repo.writable_session(branch="main")
session2 = repo.writable_session(branch="main")

root1 = zarr.group(session1.store)
root2 = zarr.group(session2.store)

root1["data"][3,:] = 3
root2["data"][4,:] = 4


session1.commit(message="Update fourth row of data array")

try:
    session2.rebase(icechunk.ConflictDetector())
    print("Rebase succeeded")
except icechunk.RebaseFailedError as e:
    print(e.conflicts)

session2.commit(message="Update fifth row of data array")

Rebase succeeded


'ADZTD57KD9V9W65DW2QG'

In [14]:
session = repo.readonly_session(branch="main")
root = zarr.open_group(session.store, mode="r")
root["data"][:,:]

array([[1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)